## Introduction

In this notebook, we will try to obtain the unique headers amongst all the csv files that we are going to merge. Then we will finally make a master comma seperated value file with list of headers that we obtain from this notebook

## Data Cleaning

### Import the necessary libaries 

In [ ]:
import pandas as pd
import numpy as np
import glob
import csv
import re
import os

### Setting up parent directory and sub directory

In [ ]:
parent_dir = "../UnifiedCSVTestData/"
filelist = []
dirs = []

def makefilelist(parent_dir):
    headers = []
    csv_headers = []
    subject_dirs = [os.path.join(parent_dir, dir) for dir in os.listdir(parent_dir) if os.path.isdir(os.path.join(parent_dir, dir))]
    filelist = []
    for dir in subject_dirs:
        csv_files = [os.path.join(dir, csv) for csv in os.listdir(dir) if os.path.isfile(os.path.join(dir, csv)) and csv.endswith('.csv')]
        for file in csv_files:
            filelist.append(file)
    
    return filelist, subject_dirs

### Read headers from CSV files

In [ ]:
def readCSV(fileList, subject_dirs):
    master_csv_headers = []
    for filename in fileList:
        slash = filename.split("/")
        parts = slash[2]
        subparts = parts.split("_")
        CIK = subparts[0]
        report_type = subparts[1]
        subsubpart = subparts[2].split('-')
        report_year = subsubpart[1]    
        df = pd.read_csv(filename,engine='python')
        df['CIK'] = CIK
        df['Reporting Type'] = report_type
        df['Report Year'] = report_year
        df.to_csv(filename, encoding='utf-8', index=False)
        
        with open(filename, 'r') as f:
            d_reader = csv.DictReader(f)
            headers = d_reader.fieldnames
            for header in headers:
                master_csv_headers.append(header)
            
    return master_csv_headers

In [ ]:
filelist, dirs = makefilelist(parent_dir)
csv_headers = readCSV(filelist, dirs)

Now we will only use the unqiue headers that we might require for our master CSV

In [ ]:
def uniqueHeaders(csv_headers):
    return set(csv_headers)

def chomp(list1):
    list1 = [x.replace('\n', '') for x in list1]
    return list1

Checking our unique header values and taking a look on the kind of values we have

In [234]:
final_headers = uniqueHeaders(csv_headers)
print(len(uniqueHeaders(csv_headers)))
lol = chomp(final_headers)
print(len(lol))

1837
1837


### Checking for different instances of similar headers

Using this to test out the best conditions required to extract all the possible variants of each header in the unified CSV

In [235]:
counter = 0
for header in lol:
    if 'fixed' in header.lower() and 'rate' in header.lower():
        print(header)
        counter+=1
print ("Counter", counter)

Pay/Receive   Fixed Rate 
Fixed       Rate
Pay/Receive Fixed     Rate
FixedRate
FixedRate
(PAY)/ RECEIVED FIXED RATE     
FixedRate(%)
(Pay) /Receive  Fixed Rate (%)  
Pay/Receive  Fixed Rate
Fixed Rate(%)
Fixed Deal Receive Rate 
(Pay)/ReceiveFixed Rate
 Fixed Rate
(PAY)/RECEIVE  FIXED RATE (%)
Fixed   Rate (%)
Fixed Rate (%) 
Receive(Pay)FixedRate
(PAY)/RECEIVE  FIXED RATE
PayFixedRate
Fixed Rate(Annualized)
 Fixed Rate(Annualized)
Fixed Rate3
Receive(Pay) FixedRate
Pay/receive fixed rate
FixedDealPay/ReceiveRate
PAY/RECEIVE FIXED RATE
ReceivedFixed Rate(Annualized)
Receive/(Pay)Fixed Rate
Fixed Rate 
Fixed Rate
FixedRate*
Fixed Rate (%)
(Pay)/Receive Fixed Rate  	
FUND PAYS/RECEIVESFIXED RATE (r) 
Fixed Rate 
(Pay)/Receive Fixed    Rate (%)
Pay/Receive  Annual  Fixed Rate
FIXED RATE
Receive/PayFixed Rate
(Pay)/ReceiveFixedRate
Fixed                  Rate
 Fixed                  Rate
Fixed Rate (%)
Receive Fixed Rate
Fixed Deal ReceiveRate
Pay Fixed Rate
(Pay)/ ReceiveFixed Rate
(Pay

### We will usse the above function as a way to guage how many similar ways are there to report a specific header and then merge all the occurances together to the master csv when we are conducting the joining

### Normalizing the values present in the column

In [ ]:
def normalizeValues(filelist):
    for filename in filelist:
        with open(filename, 'r') as f:
            df = pd.read_csv(filename, engine='python', dtype=str)
            headers = list(df)
            for header in headers:
                if '000' in header:
                    df[header] = df[header].astype(str) + '000'
            print (df)
            print("Filename", filename)        
            df.to_csv(filename, index=False, header=True)

In [ ]:
normalizeValues(filelist)

### Generate conditions for each type of Header 

In this section, we will decide the final size of the unified CSV that we are going to generate for each time user searches for something. 

In [236]:
column_headers = ["CIK", "Reporting Type", "Reporting Year", "Counterparty", "Notional Amount", "Reference Entity/Obligation", "Fixed Rate", "Expiration Date", "Appreciation/Depreciation", "Upfront Payments Paid/Received", "Implied Credit Spread", "Buy/Sell Protection", "Description"]
lol = pd.DataFrame()
CIK_csv = pd.DataFrame(columns=["CIK"])
Reporting_Type_csv = pd.DataFrame(columns=["Reporting Type"])
Reporting_Year_csv = pd.DataFrame(columns=["Reporting Year"])
Counterparty_csv = pd.DataFrame(columns=["Counterparty"])
Notional_Amount_csv = pd.DataFrame(columns=["Notional Amount"])
Reference_Entity_csv = pd.DataFrame(columns=["Reference Entity/Obligation"])
Fixed_Rate_csv = pd.DataFrame(columns=["Fixed Rate"])

#### CIK Number

In [ ]:
for filename in filelist:
    df = pd.read_csv(filename, engine='python', dtype=str)
    df = df.replace(r'\n',' ', regex=True)
    df = df.replace(r'\t',' ', regex=True) 
    headers = list(df)
    for header in headers:
        #CIK NUMBER
        if 'cik' in header.lower():
            for i in df[header].iteritems():
                CIK_csv = CIK_csv.append({'CIK': i[1]}, ignore_index=True)
        
        if 'reporting' in header.lower():
            for i in df[header].iteritems():
                Reporting_Type_csv = Reporting_Type_csv.append({'Reporting Type': i[1]}, ignore_index=True)
        
        if 'year' in header.lower():
            for i in df[header].iteritems():
                Reporting_Year_csv = Reporting_Year_csv.append({'Reporting Year': i[1]}, ignore_index=True)
        
        if 'counter' in header.lower() or 'party' in header.lower() or 'obligation' in header.lower():
            for i in df[header].iteritems():
                Counterparty_csv = Counterparty_csv.append({'Counterparty': i[1]}, ignore_index=True)
        
                
        if 'notional' in header.lower() or 'amount' in header.lower() or 'value' in header.lower():
            for i in df[header].iteritems():
                Notional_Amount_csv = Notional_Amount_csv.append({'Notional Amount': i[1]}, ignore_index=True)
                               
        if 'reference' in header.lower() or ('entity' in header.lower() or 'obligation' in header.lower()):
            for i in df[header].iteritems():
                Reference_Entity_csv = Reference_Entity_csv.append({'Reference Entity/Obligation': i[1]}, ignore_index=True)
        
        if 'fixed' in header.lower() or 'rate' in header.lower():
            for i in df[header].iteritems():
                Reference_Entity_csv = Reference_Entity_csv.append({'Fixed Rate': i[1]}, ignore_index=True)  

Let's check the final result

In [ ]:
Fixed_Rate_csv

In [ ]:
lol = CIK_csv.join(Reporting_Type_csv)
lol = lol.join(Reporting_Year_csv)
lol = lol.join(Counterparty_csv)
lol = lol.join(Notional_Amount_csv)
lol = lol.join(Reference_Entity_csv)
lol